### Initialize library

In [1]:
%load_ext autoreload
%autoreload 2

from olp_library import *
# import FOON_graph_analyser as fga

### Language to  OLP

In [10]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=True)

*************************************************************************
Stage1 Prompting
*************************************************************************
Model: gpt-4
Complete prompt:
[
    {
        "role": "system",
        "content": "You are an LLM that understands how to generate concise high level plans for arbitrary tasks involving objects. Only focus on what happens to objects.  Stay consistent with object names and use one verb per step. Assume you have all objects necessary for the plan and do not need to obtain anything."
    },
    {
        "role": "user",
        "content": "How do I make coffee?\nAfter the complete high level plan, list all the unique objects ignoring state changes to those objects. Follow the format unique_objects:object_1, obect_two, ..."
    }
]
*************************************************************************
Stage1 Response: 
1. Place coffee filter in coffee maker.
2. Add coffee grounds to coffee filter.
3. Pour water into coffee 

In [11]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Place coffee filter in coffee maker.
2. Add coffee grounds to coffee filter.
3. Pour water into coffee maker.
4. Turn on coffee maker.
5. Wait for coffee to brew.
6. Pour coffee into coffee mug.

unique_objects: coffee filter, coffee maker, coffee grounds, water, coffee mug. 

***********************
   Object level plan
***********************
{'Step': 'Place coffee filter in coffee maker', 'Objects': ['coffee filter', 'coffee maker'], 'Action': 'Place', 'StateChanges': {'coffee filter': {'Precondition': ['not in coffee maker'], 'Effect': ['in coffee maker']}, 'coffee maker': {'Precondition': ['empty'], 'Effect': ['contains coffee filter']}}}
{'Step': 'Add coffee grounds to coffee filter', 'Objects': ['coffee grounds', 'coffee filter'], 'Action': 'Add', 'StateChanges': {'coffee grounds': {'Precondition': ['not in coffee filter'], 'Effect': ['in coffee filter']}, 'coffee filter': {'Precondition': ['empty', 'in coffe

In [12]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step,plan_objects)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Place coffee filter in coffee maker
Objects: ['coffee filter', 'coffee maker']
Current object is : coffee filter
	 coffee filter state changes: {'Precondition': ['not in coffee maker'], 'Effect': ['in coffee maker']}
		 Precondition : ['not in coffee maker'] || related objects: ['coffee maker']
		 Effect : ['in coffee maker'] || related objects: ['coffee maker']
Current object is : coffee maker
	 coffee maker state changes: {'Precondition': ['empty'], 'Effect': ['contains coffee filter']}
		 Precondition : ['empty'] || related objects: []
		 Effect : ['contains coffee filter'] || related objects: ['coffee filter']
O	coffee filter	None
S	not in 	 [coffee maker]
O	coffee maker	None
M	Place	<Assumed>
O	coffee filter	None
S	in 	 [coffee maker]
O	coffee maker	None
S	contains 	{coffee filter}


In [6]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step,plan_objects)      
    new_unit.print_functions[2](version=1)
    
    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Creating functional unit for: Fill kettle with water
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['empty'], 'Effect': ['contains water']}
		 Precondition : ['empty'] || related objects: []
		 Effect : ['contains water'] || related objects: ['water']
Current object is : water
	 water state changes: {'Precondition': ['not in kettle'], 'Effect': ['in kettle']}
		 Precondition : ['not in kettle'] || related objects: ['kettle']
		 Effect : ['in kettle'] || related objects: ['kettle']
O	kettle	None
S	contains 	{water}
O	water	None
S	not in 	 [kettle]
S	in 	 [kettle]
M	Fill	<Assumed>
O	kettle	None
S	contains 	{water}
O	water	None
S	not in 	 [kettle]
S	in 	 [kettle]

Creating functional unit for: Boil water in kettle
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['contains water', 'not boiling'], 'Effect': ['contains water', 'boiling']}
		 Precondition : ['contains water', 'not boiling'] || 

In [9]:
for unit in FOON_prototype:
	unit.print_functions[2]()
	print('//')


O	kettle	None
S	contains 	{water}
O	water	None
S	not in 	 [kettle]
S	in 	 [kettle]
M	Fill	<Assumed>
O	kettle	None
S	contains 	{water}
O	water	None
S	not in 	 [kettle]
S	in 	 [kettle]
//
O	kettle	None
S	contains 	{water,water}
O	water	None
S	in 	 [kettle]
M	Boil	<Assumed>
O	kettle	None
S	contains 	{water,water}
O	water	None
S	in 	 [kettle]
//
O	coffee filter	None
S	not in 	 [coffee maker]
S	in 	 [coffee maker]
O	coffee maker	None
S	contains filter	{}
M	Place	<Assumed>
O	coffee filter	None
S	not in 	 [coffee maker]
S	in 	 [coffee maker]
O	coffee maker	None
S	contains filter	{}
//
O	coffee grounds	None
S	not in 	 [coffee filter]
S	in 	 [coffee filter]
O	coffee filter	None
S	contains 	{coffee grounds}
M	Add	<Assumed>
O	coffee grounds	None
S	not in 	 [coffee filter]
S	in 	 [coffee filter]
O	coffee filter	None
S	contains 	{coffee grounds}
//
O	kettle	None
S	contains 	{water}
O	water	None
S	in 	 [coffee maker]
S	in 	 [kettle]
O	coffee maker	None
S	contains 	{coffee grounds,coffee grounds,wate